In [1]:
from experiment import Experiment
%matplotlib inline

In [2]:
var_dict = dict(
    categorical_vars = ['Work_accident', 'promotion_last_5years', 'sales', 
                        'salary', 'number_project','time_spend_company'],
    numerical_vars = ['satisfaction_level', 'last_evaluation', 'average_montly_hours'],
    class_var = 'left'
)

In [3]:
exp = Experiment('data/HR_comma_sep.csv', 
                 var_dict,
                 embedding_dim=16,
                 max_iter=300000,
                 verbose=True,
                 n_bins_range=range(2, 21),
                 n_init_bins_list=[5, 10, 15, 20])

In [4]:
%time clf_scores = exp.classification_performance(cv=3)

>>> Iteration = 10000, Loss = 3.3297441005706787
>>> Iteration = 20000, Loss = 3.340423345565796
>>> Iteration = 30000, Loss = 3.320369243621826
>>> Iteration = 40000, Loss = 3.3405144214630127
>>> Iteration = 50000, Loss = 3.3012032508850098
>>> Iteration = 60000, Loss = 3.320502996444702
>>> Iteration = 70000, Loss = 3.411947727203369
>>> Iteration = 80000, Loss = 3.281630754470825
>>> Iteration = 90000, Loss = 3.343859910964966
>>> Iteration = 100000, Loss = 3.3821401596069336
>>> Iteration = 110000, Loss = 3.285672426223755
>>> Iteration = 120000, Loss = 3.2510015964508057
>>> Iteration = 130000, Loss = 3.3563976287841797
>>> Iteration = 140000, Loss = 3.387117862701416
>>> Iteration = 150000, Loss = 3.3305060863494873
>>> Iteration = 160000, Loss = 3.3016746044158936
>>> Iteration = 170000, Loss = 3.2639904022216797
>>> Iteration = 180000, Loss = 3.2853777408599854
>>> Iteration = 190000, Loss = 3.3493802547454834
>>> Iteration = 200000, Loss = 3.2799572944641113
>>> Iteration = 2

>>> Iteration = 150000, Loss = 3.572141170501709
>>> Iteration = 160000, Loss = 3.5493502616882324
>>> Iteration = 170000, Loss = 3.5446975231170654
>>> Iteration = 180000, Loss = 3.619056224822998
>>> Iteration = 190000, Loss = 3.6046714782714844
>>> Iteration = 200000, Loss = 3.5622997283935547
>>> Iteration = 210000, Loss = 3.5490739345550537
>>> Iteration = 220000, Loss = 3.4676783084869385
>>> Iteration = 230000, Loss = 3.590064764022827
>>> Iteration = 240000, Loss = 3.587968349456787
>>> Iteration = 250000, Loss = 3.5826382637023926
>>> Iteration = 260000, Loss = 3.510321855545044
>>> Iteration = 270000, Loss = 3.573155641555786
>>> Iteration = 280000, Loss = 3.45284104347229
>>> Iteration = 290000, Loss = 3.5756654739379883
>>> Iteration = 300000, Loss = 3.568305492401123
Learning Embedding Finished!
>>> Iteration = 10000, Loss = 3.6588847637176514
>>> Iteration = 20000, Loss = 3.6008546352386475
>>> Iteration = 30000, Loss = 3.71885085105896
>>> Iteration = 40000, Loss = 3.700

>>> Iteration = 290000, Loss = 3.604560136795044
>>> Iteration = 300000, Loss = 3.687342643737793
Learning Embedding Finished!
>>> Iteration = 10000, Loss = 3.886474609375
>>> Iteration = 20000, Loss = 3.7811028957366943
>>> Iteration = 30000, Loss = 3.7324650287628174
>>> Iteration = 40000, Loss = 3.7917721271514893
>>> Iteration = 50000, Loss = 3.851532220840454
>>> Iteration = 60000, Loss = 3.972672462463379
>>> Iteration = 70000, Loss = 3.7336418628692627
>>> Iteration = 80000, Loss = 3.740832567214966
>>> Iteration = 90000, Loss = 3.6945948600769043
>>> Iteration = 100000, Loss = 3.7929389476776123
>>> Iteration = 110000, Loss = 3.7084896564483643
>>> Iteration = 120000, Loss = 3.755544900894165
>>> Iteration = 130000, Loss = 3.754662275314331
>>> Iteration = 140000, Loss = 3.7670512199401855
>>> Iteration = 150000, Loss = 3.828596591949463
>>> Iteration = 160000, Loss = 3.7435145378112793
>>> Iteration = 170000, Loss = 3.8040876388549805
>>> Iteration = 180000, Loss = 3.750993728

In [5]:
from experiment import *

In [6]:
compute_cv_score(clf_scores['raw_scores'])

DT
Accuracy = 0.959 (+/- 0.003)

LR
Accuracy = 0.896 (+/- 0.008)



In [7]:
compute_cv_score(clf_scores['sc_scores'])

DT
Accuracy = 0.959 (+/- 0.003)

LR
Accuracy = 0.896 (+/- 0.008)



In [8]:
compute_cv_score_by_n_bins(clf_scores['ewb_scores'])

DT
#Bins = 2, Accuracy = 0.941 (+/- 0.005)
#Bins = 3, Accuracy = 0.949 (+/- 0.006)
#Bins = 4, Accuracy = 0.941 (+/- 0.002)
#Bins = 5, Accuracy = 0.937 (+/- 0.008)
#Bins = 6, Accuracy = 0.944 (+/- 0.002)
#Bins = 7, Accuracy = 0.943 (+/- 0.009)
#Bins = 8, Accuracy = 0.937 (+/- 0.003)
#Bins = 9, Accuracy = 0.938 (+/- 0.003)
#Bins = 10, Accuracy = 0.929 (+/- 0.006)
#Bins = 11, Accuracy = 0.939 (+/- 0.013)
#Bins = 12, Accuracy = 0.945 (+/- 0.015)
#Bins = 13, Accuracy = 0.930 (+/- 0.014)
#Bins = 14, Accuracy = 0.941 (+/- 0.015)
#Bins = 15, Accuracy = 0.941 (+/- 0.012)
#Bins = 16, Accuracy = 0.932 (+/- 0.006)
#Bins = 17, Accuracy = 0.934 (+/- 0.007)
#Bins = 18, Accuracy = 0.933 (+/- 0.007)
#Bins = 19, Accuracy = 0.939 (+/- 0.007)
#Bins = 20, Accuracy = 0.927 (+/- 0.006)

LR
#Bins = 2, Accuracy = 0.894 (+/- 0.010)
#Bins = 3, Accuracy = 0.912 (+/- 0.008)
#Bins = 4, Accuracy = 0.906 (+/- 0.006)
#Bins = 5, Accuracy = 0.922 (+/- 0.006)
#Bins = 6, Accuracy = 0.926 (+/- 0.002)
#Bins = 7, Accuracy = 

In [9]:
compute_cv_score_by_n_bins(clf_scores['efb_scores'])

DT
#Bins = 2, Accuracy = 0.929 (+/- 0.013)
#Bins = 3, Accuracy = 0.945 (+/- 0.005)
#Bins = 4, Accuracy = 0.926 (+/- 0.008)
#Bins = 5, Accuracy = 0.942 (+/- 0.001)
#Bins = 6, Accuracy = 0.943 (+/- 0.004)
#Bins = 7, Accuracy = 0.938 (+/- 0.004)
#Bins = 8, Accuracy = 0.939 (+/- 0.008)
#Bins = 9, Accuracy = 0.937 (+/- 0.006)
#Bins = 10, Accuracy = 0.931 (+/- 0.007)
#Bins = 11, Accuracy = 0.926 (+/- 0.001)
#Bins = 12, Accuracy = 0.931 (+/- 0.012)
#Bins = 13, Accuracy = 0.935 (+/- 0.015)
#Bins = 14, Accuracy = 0.937 (+/- 0.009)
#Bins = 15, Accuracy = 0.936 (+/- 0.007)
#Bins = 16, Accuracy = 0.943 (+/- 0.007)
#Bins = 17, Accuracy = 0.946 (+/- 0.003)
#Bins = 18, Accuracy = 0.944 (+/- 0.004)
#Bins = 19, Accuracy = 0.941 (+/- 0.003)
#Bins = 20, Accuracy = 0.943 (+/- 0.009)

LR
#Bins = 2, Accuracy = 0.892 (+/- 0.008)
#Bins = 3, Accuracy = 0.914 (+/- 0.008)
#Bins = 4, Accuracy = 0.908 (+/- 0.005)
#Bins = 5, Accuracy = 0.917 (+/- 0.009)
#Bins = 6, Accuracy = 0.925 (+/- 0.007)
#Bins = 7, Accuracy = 

In [10]:
compute_cv_score_by_n_bins(clf_scores['sb_scores'])

DT
#Bins = 5, Accuracy = 0.940 (+/- 0.007)
#Bins = 10, Accuracy = 0.935 (+/- 0.008)
#Bins = 15, Accuracy = 0.937 (+/- 0.023)
#Bins = 20, Accuracy = 0.939 (+/- 0.012)

LR
#Bins = 5, Accuracy = 0.905 (+/- 0.016)
#Bins = 10, Accuracy = 0.921 (+/- 0.019)
#Bins = 15, Accuracy = 0.932 (+/- 0.017)
#Bins = 20, Accuracy = 0.926 (+/- 0.016)

NB
#Bins = 5, Accuracy = 0.855 (+/- 0.016)
#Bins = 10, Accuracy = 0.873 (+/- 0.039)
#Bins = 15, Accuracy = 0.893 (+/- 0.013)
#Bins = 20, Accuracy = 0.886 (+/- 0.029)



### 학습된 트리의 깊이, 형태 비교

In [ ]:
dot_data = StringIO()
export_graphviz(dt, out_file=dot_data, feature_names=X.columns, 
                impurity=False, proportion=True)
graph = pydot.graph_from_dot_data(dot_data.getvalue())